In [1]:
pip install beautifulsoup4 pymysql lxml

Note: you may need to restart the kernel to use updated packages.


In [1]:
# drafter 이름 안나오는건 보완

# endYear 컬럼 추가해서 여러 기간 폴더 데이터를 하나의 테이블에 구분해서 저장할 수 있게 변경

# + type이 합의인것만 - AGREEMENT로 변경
# 디비 저장 오류 수정 

import os
import json
import re
from datetime import datetime
from bs4 import BeautifulSoup
import pymysql
from pathlib import Path

class ApprovalDocParser:
    def __init__(self, base_path):
        self.base_path = base_path
        
    def extract_source_id(self, filename):
        """파일명에서 마지막 숫자 추출"""
        numbers = re.findall(r'\d+', filename)
        return numbers[-1] if numbers else None
    
    def parse_datetime_to_unix(self, date_str):
        """날짜 문자열을 Unix timestamp(밀리초)로 변환"""
        try:
            dt = datetime.strptime(date_str.strip(), "%Y-%m-%d %H:%M:%S")
            return int(dt.timestamp() * 1000)
        except:
            return None
    
    def extract_person_info(self, text):
        """이름/직책/부서 형식에서 정보 추출"""
        text = re.sub(r'\d+', '', text).strip()
        parts = text.split('/')
        if len(parts) >= 3:
            return {
                'name': parts[0].strip(),
                'positionName': parts[1].strip(),
                'deptName': parts[2].strip()
            }
        return None
    
    def parse_html(self, html_path):
        """HTML 파일 파싱"""
        with open(html_path, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f.read(), 'html.parser')
        
        filename = os.path.basename(html_path)
        source_id = self.extract_source_id(filename)
        
        doc_num_elem = soup.find('p', class_='team')
        doc_num = doc_num_elem.get_text(strip=True) if doc_num_elem else ""
        
        references_elem = soup.find('span', class_='disInline o-i-min-fileList')
        references = []
        if references_elem:
            next_elem = references_elem.find_next_sibling()
            if next_elem:
                references = next_elem.get_text(strip=True)
            else:
                parent = references_elem.parent
                references = parent.get_text(strip=True)
        
        title_elem = soup.find('title')
        title = title_elem.get_text(strip=True) if title_elem else ""
        
        attaches = []
        verti_tops = soup.find_all('span', class_='verti_Top')
        for vt in verti_tops:
            a_tag = vt.find('a')
            if a_tag and a_tag.get('href'):
                attaches.append({
                    'name': a_tag.get_text(strip=True),
                    'path': a_tag.get('href')
                })
        
        # 기존 방식: user_spans에서 drafter 추출
        user_spans = soup.find_all('span', class_='user')
        drafter = {}
        if user_spans:
            first_user = user_spans[0]
            name_elem = first_user.find('span', class_='F_12_black_b')
            if name_elem:
                info = self.extract_person_info(name_elem.get_text(strip=True))
                if info:
                    drafter = {
                        'positionName': info['positionName'],
                        'deptName': info['deptName'],
                        'name': info['name'],
                        'emailId': '',
                        'deptCode': ''
                    }
        
        # drafter가 비어있으면 테이블의 "기안자" 행에서 추출 (백업)
        if not drafter or not drafter.get('name'):
            drafter_th = soup.find('th', string=lambda s: s and '기안자' in s)
            if drafter_th:
                drafter_td = drafter_th.find_next_sibling('td')
                if drafter_td:
                    bg01_div = drafter_td.find('div', class_='bg01')
                    if bg01_div:
                        name = bg01_div.get_text(strip=True)
                        if name:  # 이름이 실제로 있으면
                            drafter = {
                                'positionName': '',
                                'deptName': '',
                                'name': name,
                                'emailId': '',
                                'deptCode': ''
                            }
        
        created_at = None
        if user_spans:
            date_elem = user_spans[0].find('span', class_='F_11_gray')
            if date_elem:
                created_at = self.parse_datetime_to_unix(date_elem.get_text(strip=True))
        
        referrers = []
        ref_th = soup.find('th', string=lambda s: s and '참조' in s)
        if ref_th:
            ref_td = ref_th.find_next_sibling('td')
            if ref_td:
                ref_text = ref_td.get_text(strip=True)
                ref_list = re.split(r'[,\s]+', ref_text)
                for ref in ref_list:
                    if ref:
                        info = self.extract_person_info(ref)
                        if info:
                            referrers.append({
                                'name': info['name'],
                                'empNo': '',
                                'deptCode': ''
                            })
        
        # 결재선 영역에서 "합의"인 사람들의 이름 추출
        agreement_names = []
        appr_line_area = soup.find('div', id='apprLineArea')
        if appr_line_area:
            red_spans = appr_line_area.find_all('span', class_='red')
            for red_span in red_spans:
                if '합의' in red_span.get_text(strip=True):
                    # red span의 부모 li의 이전 형제 li에서 이름 찾기
                    parent_li = red_span.find_parent('li')
                    if parent_li:
                        prev_li = parent_li.find_previous_sibling('li')
                        if prev_li:
                            name_text = prev_li.get_text(strip=True)
                            # 이름만 추출 (숫자 제거 등)
                            if name_text:
                                agreement_names.append(name_text)
        
        activities = []
        for idx, user_span in enumerate(user_spans):
            name_elem = user_span.find('span', class_='F_12_black_b')
            date_elem = user_span.find('span', class_='F_11_gray')
            
            if name_elem:
                info = self.extract_person_info(name_elem.get_text(strip=True))
                if info:
                    action_comment = ""
                    next_elem = user_span.next_sibling
                    while next_elem:
                        if hasattr(next_elem, 'name') and next_elem.name == 'div':
                            action_comment = next_elem.get_text(strip=True)
                            break
                        next_elem = next_elem.next_sibling
                    
                    # 기본 타입 설정
                    action_type = 'DRAFT' if idx == 0 else 'APPROVAL'
                    
                    # 이 사람이 합의자 리스트에 있으면 AGREEMENT로 변경
                    if info['name'] in agreement_names:
                        action_type = 'AGREEMENT'
                    
                    activities.append({
                        'positionName': info['positionName'],
                        'deptName': info['deptName'],
                        'actionLogType': action_type,
                        'name': info['name'],
                        'emailId': '',
                        'type': action_type,
                        'actionDate': self.parse_datetime_to_unix(date_elem.get_text(strip=True)) if date_elem else None,
                        'deptCode': '',
                        'actionComment': action_comment
                    })
        
        result = {
            'sourceId': source_id,
            'docNum': doc_num,
            'references': references,
            'docType': 'DRAFT',
            'title': title,
            'attaches': attaches,
            'drafter': drafter,
            'createdAt': created_at,
            'docBody': '',
            'docStatus': 'COMPLETE',
            'referrers': referrers,
            'activities': activities,
            'formName': '',
            'isPublic': False
        }
        
        return result
    
    def process_all_files(self):
        """모든 HTML 파일 처리"""
        all_results = []
        approval_path = Path(self.base_path) / '결재'
        
        if not approval_path.exists():
            print(f"경로를 찾을 수 없습니다: {approval_path}")
            return all_results
            
        html_files = list(approval_path.rglob('*.html'))
        # html_files = list(approval_path.rglob('*.html'))[:10]
        print(f"총 {len(html_files)}개의 HTML 파일을 찾았습니다.")
        
        for idx, html_file in enumerate(html_files, 1):
            try:
                print(f"처리 중... [{idx}/{len(html_files)}] {html_file.name}")
                result = self.parse_html(html_file)
                all_results.append(result)
            except Exception as e:
                print(f"오류 발생 ({html_file.name}): {e}")
        
        return all_results
    
    def save_to_json(self, data, output_path):
        """JSON 파일로 저장"""
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"JSON 파일 저장 완료: {output_path}")
    
    def save_to_mariadb(self, data, db_config, end_year):
        """MariaDB에 저장 - 안정성 강화 버전"""
        conn = None
        cursor = None
        
        try:
            print("\n=== DB 연결 시작 ===")
            print(f"Host: {db_config['host']}, Database: {db_config['database']}")
            
            # DB 연결
            conn = pymysql.connect(
                host=db_config['host'],
                user=db_config['user'],
                password=db_config['password'],
                database=db_config['database'],
                charset='utf8mb4'
            )
            print("✓ DB 연결 성공")
            
            cursor = conn.cursor()
            
            # 테이블 생성
            print("\n=== 테이블 생성 시작 ===")
            create_table_sql = """
            CREATE TABLE IF NOT EXISTS documents (
                id INT AUTO_INCREMENT PRIMARY KEY,
                source_id VARCHAR(50),
                doc_num VARCHAR(100),
                doc_type VARCHAR(50),
                title VARCHAR(500),
                doc_status VARCHAR(50),
                created_at BIGINT,
                drafter_name VARCHAR(100),
                drafter_position VARCHAR(100),
                drafter_dept VARCHAR(100),
                form_name VARCHAR(200),
                is_public BOOLEAN,
                end_year INT,
                `references` TEXT,
                attaches TEXT,
                referrers TEXT,
                activities TEXT,
                doc_body TEXT,
                created_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                INDEX idx_source_id (source_id),
                INDEX idx_doc_num (doc_num),
                INDEX idx_created_at (created_at),
                INDEX idx_end_year (end_year)
            ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
            """
            cursor.execute(create_table_sql)
            print("✓ 테이블 생성/확인 완료")
            
            # 기존 데이터 확인
            cursor.execute("SELECT COUNT(*) FROM documents")
            existing_count = cursor.fetchone()[0]
            print(f"✓ 기존 데이터: {existing_count}건")
            
            # 데이터 삽입
            print(f"\n=== 데이터 삽입 시작 ({len(data)}건) ===")
            insert_sql = """
            INSERT INTO documents 
            (source_id, doc_num, doc_type, title, doc_status, created_at, 
             drafter_name, drafter_position, drafter_dept, form_name, is_public, end_year,
             `references`, attaches, referrers, activities, doc_body)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            
            success_count = 0
            error_count = 0
            error_details = []
            
            for idx, doc in enumerate(data, 1):
                try:
                    # 데이터 길이 제한 및 안전한 변환
                    def safe_string(value, max_length=None):
                        if value is None:
                            return ''
                        s = str(value).strip()
                        if max_length and len(s) > max_length:
                            s = s[:max_length]
                        return s
                    
                    # JSON 데이터를 안전하게 변환 (ensure_ascii=True로 특수문자 이스케이프)
                    def safe_json(value):
                        if not value:
                            return '[]'
                        try:
                            # ensure_ascii=True로 모든 비ASCII 문자를 이스케이프
                            return json.dumps(value, ensure_ascii=True)
                        except:
                            return '[]'
                    
                    values = (
                        safe_string(doc.get('sourceId'), 50),
                        safe_string(doc.get('docNum'), 100),
                        safe_string(doc.get('docType'), 50),
                        safe_string(doc.get('title'), 500),
                        safe_string(doc.get('docStatus'), 50),
                        doc.get('createdAt'),
                        safe_string(doc.get('drafter', {}).get('name'), 100),
                        safe_string(doc.get('drafter', {}).get('positionName'), 100),
                        safe_string(doc.get('drafter', {}).get('deptName'), 100),
                        safe_string(doc.get('formName'), 200),
                        doc.get('isPublic', False),
                        end_year,
                        safe_string(doc.get('references')),
                        safe_json(doc.get('attaches', [])),
                        safe_json(doc.get('referrers', [])),
                        safe_json(doc.get('activities', [])),
                        safe_string(doc.get('docBody'))
                    )
                    
                    cursor.execute(insert_sql, values)
                    success_count += 1
                    
                    # 100건마다만 출력 (로그 줄이기)
                    if idx % 100 == 0 or idx == len(data):
                        print(f"  진행: {idx}/{len(data)} (성공: {success_count}, 실패: {error_count})")
                    
                except Exception as e:
                    error_count += 1
                    error_msg = f"sourceId: {doc.get('sourceId')} - {str(e)[:80]}"
                    error_details.append(error_msg)
                    
                    # 처음 5개 오류만 즉시 출력
                    if error_count <= 5:
                        print(f"  [오류 {error_count}] {error_msg}")
            
            # 커밋
            print("\n=== 커밋 시작 ===")
            conn.commit()
            print(f"✓ 커밋 완료")
            
            # 결과 요약
            print(f"\n=== 결과 요약 ===")
            print(f"✓ 성공: {success_count}건")
            print(f"✗ 실패: {error_count}건")
            
            if error_count > 5:
                print(f"\n처음 5개 외 {error_count - 5}개의 추가 오류가 발생했습니다.")
                print("모든 오류 보기:")
                for err in error_details[:20]:  # 최대 20개만 출력
                    print(f"  - {err}")
                if len(error_details) > 20:
                    print(f"  ... 외 {len(error_details) - 20}개 더")
            
            # 최종 확인
            cursor.execute("SELECT COUNT(*) FROM documents")
            final_count = cursor.fetchone()[0]
            print(f"\n✓ 최종 데이터: {final_count}건 (이번 실행으로 {success_count}건 추가)")
            
        except pymysql.Error as e:
            print(f"\n❌ DB 오류 발생:")
            print(f"  Error Code: {e.args[0]}")
            print(f"  Error Message: {e.args[1]}")
            
        except Exception as e:
            print(f"\n❌ 일반 오류 발생: {e}")
            
        finally:
            if cursor:
                cursor.close()
            if conn:
                conn.close()
                print("\n✓ DB 연결 종료")


def main():
    base_path = r'C:\Users\LEEJUHWAN\Downloads\2010-01-01~2010-12-31\html' #컨트롤러
    end_year = 2010  #컨트롤러 - 폴더 기간에 맞게 수정
    
    parser = ApprovalDocParser(base_path)
    
    print("HTML 파일 파싱 시작...")
    results = parser.process_all_files()
    
    output_json_path = 'two_htmltojsondb_convert.json' #컨트롤러
    parser.save_to_json(results, output_json_path)
    
    # DB 설정
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'password': '1234',
        'database': 'any_approval' #컨트롤러 - 통합 DB명
    }
    
    # MariaDB에 저장
    print("\n" + "="*50)
    parser.save_to_mariadb(results, db_config, end_year)
    print("="*50)
    
    print(f"\n완료! 총 {len(results)}건 처리됨")


if __name__ == '__main__':
    main()

HTML 파일 파싱 시작...
총 840개의 HTML 파일을 찾았습니다.
처리 중... [1/840] 20100108_국내출장비 신청합니다._2002068.html
처리 중... [2/840] 20100108_출장비 신청합니다._2002067.html
처리 중... [3/840] 20100109_출장비_2002069.html
처리 중... [4/840] 20100118_국내출장비 신청(창원대,인제대,실트론)_2002070.html
처리 중... [5/840] 20100118_사진보드 꾸밀시 필요한  비품 신청_2002071.html
처리 중... [6/840] 20100118_사진보드 꾸밀시 필요한  비품 신청_2002072.html
처리 중... [7/840] 20100118_한국한의학 연구원 프로젝트 인원투입요청 (2010.01.18)_2002073.html
처리 중... [8/840] 20100119_[기술연구부문] 회사 공용 개발서버 Memory 확장관련 비용승인 품의_2002074.html
처리 중... [9/840] 20100120_USB메모리 구매신청_2002075.html
처리 중... [10/840] 20100120_[채용품의] ITO 팀 아웃소싱전문가 채용_2002078.html
처리 중... [11/840] 20100120_[회사공통 영업용 Demo 솔루션 관리를 위한 USB구매요청]_2002076.html
처리 중... [12/840] 20100120_출장비용 품의 드립니다._2002077.html
처리 중... [13/840] 20100121_LG하우시스 개발자 김재호씨 업무 중지 요청입니다._2002079.html
처리 중... [14/840] 20100122_[견적품의] 식품의약품안전평가원 연구관리시스템 등 운영을 위한 동부CNI 견적 제출건_2002082.html
처리 중... [15/840] 20100122_[채용품의] 식품의약품안전평가원 연구관리시스템 운영인력_2002081.html
처리 중... [16/840] 20100122